In [ ]:
!python -m spacy download en_core_web_lg
#nlp = spacy.load("en_core_web_lg")
!pip install sentence-transformers
!pip install gensim
!pip install networks
!pip install matplotlib
!pip install yake
!pip install editdistance==0.3.1

2021-03-22 01:14:34.192794: W tensorflow/stream_executor/platform/default/dso_loader.cc:60] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-03-22 01:14:34.192833: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
You should consider upgrading via the '/root/venv/bin/python -m pip install --upgrade pip' command.
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')
     |████████████████████████████████| 74 kB 6.5 MB/s 
     |████████████████████████████████| 2.0 MB 39.0 MB/s 
     |████████████████████████████████| 1.2 MB 66.4 MB/s 
     |████████████████████████████████| 883 kB 65.6 MB/s 
     |████████████████████████████████| 3.2 MB 55.4 MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-1.0.2-py3-none-any.whl size=114269 sha256=2ef6418

In [ ]:
import pandas as pd

## Named entities

    ORG = Organization,companies,agencies, institutions, etc.
    PER = People, including fictional.
    LOC = Non-GPE locations, mountain ranges, bodies of water.
    MISC = Miscellaneous entities, e.g. events, nationalities, products or works of art

In [ ]:
#pd.read_pickle("/work/News_dataset/pd_news_dataset_2.pickle")

# Measure keyword extractor quality

In [ ]:
import sys
sys.path.insert(1, '/work/IFT6010_Story_Tree/src/features/')

In [ ]:
from keyword_extractor import f_measure
from keyword_extractor import keyword_extractor
from keyword_extractor import keyword_extraction_baseline
from keyword_extractor import keyword_extraction_baseline_gensim

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
articles_data_keyword = pd.read_pickle("../../Keyword_dataset/dataset_with_extracted_keyword.pickle")

In [ ]:
average_precision = []

for i in range(len(articles_data_keyword)):
    output = f_measure(articles_data_keyword.iloc[i]['keyword_extracted'],articles_data_keyword.iloc[i]['keywords_label'])
    average_precision.append(output[1])

print(sum(average_precision)/len(average_precision))

0.6960499096590558


In [ ]:
average_precision = []

for i in range(len(articles_data_keyword)):
    output = f_measure(articles_data_keyword.iloc[i]['keyword_extracted'],articles_data_keyword.iloc[i]['keywords_label'])
    average_precision.append(output[2])

print(sum(average_precision)/len(average_precision))

0.5338507679148621


In [ ]:
articles_data_keyword["keyword_extracted_baseline"] = articles_data_keyword["text"].apply(keyword_extraction_baseline)
#articles_data_keyword.to_pickle("dataset_with_extracted_keyword.pickle")

In [ ]:
articles_data_keyword["keyword_extraction_baseline_gensim"] = articles_data_keyword["text"].apply(keyword_extraction_baseline_gensim)

In [ ]:
avg = 0
for i in range(len(articles_data_keyword)):
    avg += len(articles_data_keyword.iloc[i]["keyword_extraction_baseline_gensim"])

print(avg/len(articles_data_keyword))

31.32888888888889


In [ ]:
avg = 0
for i in range(len(articles_data_keyword)):
    avg += len(articles_data_keyword.iloc[i]["keywords_label"])

print(avg/len(articles_data_keyword))

49.56666666666667


In [ ]:
avg = 0
for i in range(len(articles_data_keyword)):
    avg += len(articles_data_keyword.iloc[i]["keyword_extracted_baseline"])

print(avg/len(articles_data_keyword))

30.45777777777778


In [ ]:
avg = 0
for i in range(len(articles_data_keyword)):
    avg += len(articles_data_keyword.iloc[i]["keyword_extracted"])

print(avg/len(articles_data_keyword))

33.47555555555556


In [ ]:
average_precision = []

for i in range(len(articles_data_keyword)):
    output = f_measure(articles_data_keyword.iloc[i]['keyword_extraction_baseline_gensim'],articles_data_keyword.iloc[i]['keywords_label'])
    average_precision.append(output[1])

print(sum(average_precision)/len(average_precision))

0.6833529709420512


In [ ]:
average_precision = []

for i in range(len(articles_data_keyword)):
    output = f_measure(articles_data_keyword.iloc[i]['keyword_extraction_baseline_gensim'],articles_data_keyword.iloc[i]['keywords_label'])
    average_precision.append(output[2])

print(sum(average_precision)/len(average_precision))

0.48356496935549415


In [ ]:
average_precision = []

for i in range(len(articles_data_keyword)):
    output = f_measure(articles_data_keyword.iloc[i]['keyword_extracted_baseline'],articles_data_keyword.iloc[i]['keywords_label'])
    average_precision.append(output[1])

print(sum(average_precision)/len(average_precision))

KeyError: 'keyword_extracted_baseline'

In [ ]:
average_precision = []

for i in range(len(articles_data_keyword)):
    output = f_measure(articles_data_keyword.iloc[i]['keyword_extracted_baseline'],articles_data_keyword.iloc[i]['keywords_label'])
    average_precision.append(output[2])

print(sum(average_precision)/len(average_precision))

0.5509106083480148


# Random stuff

In [ ]:
#articles_data_keyword = pd.read_pickle("keyword_dataset.pickle")
#articles_data_keyword["keyword_extracted"] = articles_data_keyword["text"].apply(keyword_extractor)
#articles_data_keyword.to_pickle("dataset_with_extracted_keyword.pickle")

#news_dataset = pd.read_pickle("/work/News_dataset/pd_news_dataset_2.pickle")
#news_dataset["extracted_keywords"] = news_dataset["TEXT"].apply(keyword_extractor)
#news_dataset.to_pickle("/work/News_dataset/pd_news_dataset_2.pickle")

### TEST ARTICLES

In [ ]:
import pickle
import pandas as pd
import numpy as np
import re
import nltk
import itertools
import yake
import gensim
import spacy

from sklearn.feature_extraction.text import CountVectorizer
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer
from nltk.tokenize import word_tokenize
from gensim.summarization import keywords as keywords_gensim
from gensim.summarization.keywords import get_graph
import networkx as nx
import matplotlib.pyplot as plt
import editdistance as Levenshtein

nltk.download('stopwords')
nltk.download('punkt')

stop_words = set(stopwords.words('english'))
stemmer = SnowballStemmer("english")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
articles = pd.read_csv("News_dataset/short_articles_all_the_news.csv", header=1, names=["id", "title", "publication", "author", "date", "year", "month", "url", "content"], warn_bad_lines=True, error_bad_lines=False)

FileNotFoundError: [Errno 2] No such file or directory: 'News_dataset/short_articles_all_the_news.csv'

In [ ]:
keyword_list = keyword_extractor(articles.loc[0]["content"],10)

In [ ]:
#keyword_list

In [ ]:
articles.loc[0]["content"]

'After the bullet shells get counted, the blood dries and the votive candles burn out, people peer down from   windows and see crime scenes gone cold: a band of yellow police tape blowing in the breeze. The South Bronx, just across the Harlem River from Manhattan and once shorthand for urban dysfunction, still suffers violence at levels long ago slashed in many other parts of New York City. And yet the city’s efforts to fight it remain splintered, underfunded and burdened by scandal. In the 40th Precinct, at the southern tip of the Bronx, as in other poor, minority neighborhoods across the country, people long hounded for   infractions are crying out for more protection against grievous injury or death. By September, four of every five shootings in the precinct this year were unsolved. Out of the city’s 77 precincts, the 40th has the highest murder rate but the fewest detectives per violent crime, reflecting disparities in staffing that hit hardest in some neighborhoods outside Manhatt

### TEST DSK ARTICLES

In [ ]:
articles_dsk = pd.read_csv("dsk_articles.csv", header=1, names=['title', 'summary','text','date'], warn_bad_lines=True, error_bad_lines=False)

In [ ]:
keyword_list_dsk = keyword_extractor(articles_dsk.loc[10]["text"],10)

In [ ]:
len(keyword_list_dsk)

41

### Preprocessing Facebook

In [ ]:
train_set = pickle.load(open("shorter_train_set.pickle", "rb"))

In [ ]:
qus_list = []
qus_with_code = 0
len_before_preprocessing = 0 
len_after_preprocessing = 0 
for index,row in train_set.iterrows():
    title, body, tags = row["Title"], row["Body"], row["Tags"]
    if '<code>' in body:
        qus_with_code+=1
    len_before_preprocessing+=len(title) + len(body)
    body=re.sub('<code>(.*?)</code>', '', body, flags=re.MULTILINE|re.DOTALL)
    body = re.sub('<.*?>', ' ', str(body.encode('utf-8')))
    title=title.encode('utf-8')
    question=str(title)+" "+str(body)
    question=re.sub(r'[^A-Za-z]+',' ',question)
    words=word_tokenize(str(question.lower()))
    question=' '.join(str(stemmer.stem(j)) for j in words if j not in stop_words and (len(j)!=1 or j=='c'))
    qus_list.append(question)
    len_after_preprocessing += len(question)
train_set["question"] = qus_list
avg_len_before_preprocessing=(len_before_preprocessing*1.0)/train_set.shape[0]
avg_len_after_preprocessing=(len_after_preprocessing*1.0)/train_set.shape[0]
print( "Avg. length of questions(Title+Body) before preprocessing: ", avg_len_before_preprocessing)
print( "Avg. length of questions(Title+Body) after preprocessing: ", avg_len_after_preprocessing)
print ("% of questions containing code: ", (qus_with_code*100.0)/train_set.shape[0])

### Clean data set and get 60 random articles

In [ ]:
# import pandas as pd
from random import seed
from random import randint

articles = pd.read_csv("articles1.csv", header=1, names=["id", "title", "publication", "author", "date", "year", "month", "url", "content"], warn_bad_lines=True, error_bad_lines=False)
articles_drop = articles.dropna(subset=['content', 'title','date'])
articles_random = articles_drop.sample(n=60, random_state=1)

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=902703f2-430f-48f3-ba3f-6c2fee66cf11' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>